Code for scrapping

In [1]:
import requests
from bs4 import BeautifulSoup


def get_html_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup.prettify()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"Request exception occurred: {req_err}")
    except Exception as err:
        print(f"An error occurred: {err}")
    return None

In [2]:
# url = 'https://drt.etribunals.gov.in/drtlive/Misdetailreport.php?no=MDkwMTIwMDA5MTgyMDIzL2x1Y2tub3c='
url = "https://drt.etribunals.gov.in/drtlive/Misdetailreport.php?no=MDkwMTIwMDAwODUyMDIyL2x1Y2tub3c="
html_ = get_html_from_url(url)
# print(html_soup)
with open('code.html', '+w') as f:
    f.write(str(html_))
    f.close()

soup = BeautifulSoup(html_, 'html.parser')
# print(soup.prettify())
# print(soup.text)

In [3]:
import re
from unicodedata import normalize


def normalize_text(raw_text):
    return re.sub("\s\s+", " ", normalize("NFKD", raw_text.strip()))

In [4]:
def splitMystirng(text):
    lSting = text.split('\n')
    tempL = []
    for i in lSting:
        tempS = i.strip()
        if tempS != "":
            tempL.append(tempS)
    return tempL

In [5]:
table = soup.find(class_="table table-bordered table-extra-condensed")
table2RawString = normalize_text(table.text)
print(table2RawString)

# for case proceeding detail
tempTable = soup.find_all(class_="table table-bordered")
for i in tempTable:
    if i.find_all("tr") and normalize_text(i.find_all("tr")[0].text) == "CASE PROCEEDING DETAILS":
        caseProceedingTable = i
for i in caseProceedingTable.find_all('td'):
    if normalize_text(i.text) == "RC/TRC CASE CURRENT STATUS":
        break
    print(normalize_text(i.text))

CASE STATUS Diary no/Year 85/2022 Case Type/Case No/Year Original Application/50/2022 Date of Filing. 13/01/2022 Case Status. Pending CASE LISTING DETAILS (Tentative) In the Court of PO Court No. 1 Next Listing Date 10/08/2023 Next Listing Purpose APPEARANCE PETITIONER/APPLICANT DETAIL Petitioner Name -STATE BANK OF INDIA Petitioner/Applicant Address: SARB II 18 4 4TH FLOOR SBI HOUSE ARYA SAMAJ ROAD KAROL BAGH NEW DELHI Additional Party: Advocate Name: PRASHANT KUMAR SRIVASTAVAAdditional Advocate: RESPONDENTS/DEFENDENT DETAILS Respondent Name -VIVEK UPRAITY Respondent/Defendent Address: Additional Party: RUCHI TOMAR ,ANSAL URBAN CONDOMINIUMS , Advocate Name - Additional Advocate: PROPERTY DETAILS Property Type Detail Of Property IMMOVABLE FLAT NO ,02/10 D 02 ,ANSAL AQUAPOLI GHAZIABAD UP , CASE PROCEEDING DETAILS Court Name Causelist Date Purpose Registrar 10/02/2023 SERVICE STAGE PO 07/02/2023 APPEARANCE PO 20/10/2022 APPEARANCE PO 14/07/2022 APPEARANCE Registrar 22/03/2022 SERVICE STA

In [6]:
tempTable = soup.find_all(class_="table table-striped")
for i in tempTable:
    if i.find_all("tr") and normalize_text(i.find_all("tr")[0].text) == "PROPERTY DETAILS":
        caseProceedingTable = i
for i in caseProceedingTable.find_all('td'):
    print(normalize_text(i.text))

PROPERTY DETAILS
Property Type
Detail Of Property
IMMOVABLE
FLAT NO ,02/10 D 02 ,ANSAL AQUAPOLI GHAZIABAD UP ,


In [7]:
import simplematch
boundry = [
    "CASE STATUS {caseStatus} ",
    "CASE LISTING DETAILS (Tentative) {caseDetail} ",
    "PETITIONER/APPLICANT DETAIL {petDetail}",
    "RESPONDENTS/DEFENDENT DETAILS {resDetail}",
    "PROPERTY DETAILS {propertyDetailList}",
    "CASE PROCEEDING DETAILS {caseProceedingDetaillist}",
    "RC/TRC CASE CURRENT STATUS {rcTcsCaseCurrentStatus}",
    "RC CASE PROCEEDING DETAILS {rcCaseProceedingDetailList}",
]


def add_string_list(string_list):
    result = ""
    for string in string_list:
        result += string
    return result


dataByBoundry = simplematch.match(add_string_list(boundry), table2RawString)

# print(add_string_list(TABLE_CASE_STATUS_KEYS))
dataByBoundry

{'caseStatus': 'Diary no/Year 85/2022 Case Type/Case No/Year Original Application/50/2022 Date of Filing. 13/01/2022 Case Status. Pending',
 'caseDetail': 'In the Court of PO Court No. 1 Next Listing Date 10/08/2023 Next Listing Purpose APPEARANCE',
 'petDetail': 'Petitioner Name -STATE BANK OF INDIA Petitioner/Applicant Address: SARB II 18 4 4TH FLOOR SBI HOUSE ARYA SAMAJ ROAD KAROL BAGH NEW DELHI Additional Party: Advocate Name: PRASHANT KUMAR SRIVASTAVAAdditional Advocate: ',
 'resDetail': 'Respondent Name -VIVEK UPRAITY Respondent/Defendent Address: Additional Party: RUCHI TOMAR ,ANSAL URBAN CONDOMINIUMS , Advocate Name - Additional Advocate: ',
 'propertyDetailList': 'Property Type Detail Of Property IMMOVABLE FLAT NO ,02/10 D 02 ,ANSAL AQUAPOLI GHAZIABAD UP , ',
 'caseProceedingDetaillist': 'Court Name Causelist Date Purpose Registrar 10/02/2023 SERVICE STAGE PO 07/02/2023 APPEARANCE PO 20/10/2022 APPEARANCE PO 14/07/2022 APPEARANCE Registrar 22/03/2022 SERVICE STAGE PO 15/03/202

In [8]:
caseStatusKeys = [
    "Diary no/Year {diary_number} ",
    "Case Type/Case No/Year {case_type_number_year} ",
    "Date of Filing. {filing_date} ",
    "Case Status. {case_status} ",
    "Main Case -Diary No. {main_case_dairy_num} ",
    "Main Case - Case No. {main_case_num}",
]

caseDetailkeys = [
    "In the Court of {court_of} ",
    "Court No. {court_num} ",
    "Next Listing Date {next_listing_date} ",
    "Next Listing Purpose {next_listing_purpose}"
]

petDetailKeys = [
    "Petitioner Name -{pname} "
    "Petitioner/Applicant Address:{pet_address} ",
    "Additional Party: {additional_pet_party} ",
    "Advocate Name: {pet_adv}",
    "Additional Advocate: {additional_pet_adv}"
]

resDetailKeys = ["Respondent Name -{rname} ",
                 "Respondent/Defendent Address:{res_address} ",
                 "Additional Party: {add} ",
                 "Advocate Name -{respondent_advocates} ",
                 "Additional Advocate: {additional_res_adv}"
                 ]
rcTcsCseCurrentStatuskey = [
    "Court Name: -{courtName} ",
    "Next Listing Date: {nextListingDate} ",
    "Next Listing Purpose: {nextListingPurpose}"
]


def detailstrToList(keyStr, keyDict, detListStr):
    keyPattern = keyStr[0]
    for p in keyStr[1:]:
        keyPattern = keyPattern + " " + p
    keyPattern = keyPattern+" {data}"
    print(keyPattern)
    print(dataByBoundry['caseProceedingDetaillist'])
    temp_i = 0
    temp_l = simplematch.match(keyPattern, detListStr)['data'].split()
    output = []

    col_no = len(keyStr)
    temp_i = 0
    tempDict = {}
    for v in temp_l:
        if temp_i < col_no:
            tempDict[keyDict[temp_i]] = v
            temp_i = temp_i+1
        else:
            output.append(tempDict)
            temp_i = 0
            tempDict = {}
    return output


keysforcaseProceedingDetaillist = [
    "Court Name", "Causelist Date", "Purpose"]
keysfordictcaseProceedingDetaillist = [
    "court_name", "couselist_date", "purpose"
]


detailstrToList(keysforcaseProceedingDetaillist,
                keysfordictcaseProceedingDetaillist, dataByBoundry['caseProceedingDetaillist'])

Court Name Causelist Date Purpose {data}
Court Name Causelist Date Purpose Registrar 10/02/2023 SERVICE STAGE PO 07/02/2023 APPEARANCE PO 20/10/2022 APPEARANCE PO 14/07/2022 APPEARANCE Registrar 22/03/2022 SERVICE STAGE PO 15/03/2022 APPEARANCE Registrar 04/03/2022 FRESH MATTERS 


[{'court_name': 'Registrar',
  'couselist_date': '10/02/2023',
  'purpose': 'SERVICE'},
 {'court_name': 'PO', 'couselist_date': '07/02/2023', 'purpose': 'APPEARANCE'},
 {'court_name': '20/10/2022', 'couselist_date': 'APPEARANCE', 'purpose': 'PO'},
 {'court_name': 'APPEARANCE',
  'couselist_date': 'Registrar',
  'purpose': '22/03/2022'},
 {'court_name': 'STAGE', 'couselist_date': 'PO', 'purpose': '15/03/2022'},
 {'court_name': 'Registrar',
  'couselist_date': '04/03/2022',
  'purpose': 'FRESH'}]

In [ ]:
[{'court_name': 'PO', 'couselist_date': '08/09/2022', 'purpose': 'JUDGMENT'}, {'court_name': '21/07/2022', 'couselist_date': 'APPEARANCE', 'purpose': 'PO'},
    {'court_name': 'APPEARANCE', 'couselist_date': 'PO', 'purpose': '07/04/2022'}, {'court_name': 'PO', 'couselist_date': '16/03/2022', 'purpose': 'APPEARANCE'}]
[{'court_name': 'PO', 'couselist_date': '08/09/2022', 'purpose': 'JUDGMENT'}, {'court_name': '21/07/2022', 'couselist_date': 'APPEARANCE', 'purpose': 'PO'},
    {'court_name': 'APPEARANCE', 'couselist_date': 'PO', 'purpose': '07/04/2022'}, {'court_name': 'PO', 'couselist_date': '16/03/2022', 'purpose': 'APPEARANCE'}]

In [27]:
data1 = {'drt-ahmedabad-1': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-ahmedabad-2': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-allahabad': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-aurangabad': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-bangalore-1': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-bangalore-2': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-chandigarh-1': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-chandigarh-2': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-chandigarh-3': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-chennai-1': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-chennai-2': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-chennai-3': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-coimbatore': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-cuttack': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-dehradun': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-delhi-1': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-delhi-2': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-delhi-3': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-ernakulam-1': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-ernakulam-2': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-guwahati': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-hyderabad-1': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-hyderabad-2': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-jabalpur': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}, {'name': 'OA.EX/T.EX/TA.EX', 'value': '7777'}], 'drt-jaipur': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-kolkata-1': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-kolkata-2': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-kolkata-3': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-lucknow': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-madurai': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-mumbai-1': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-mumbai-2': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-mumbai-3': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-nagpur': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-patna': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-pune': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-ranchi': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-siliguri': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}], 'drt-vishakhapatnam': [{'name': 'RC', 'value': '99999'}, {'name': 'TRC', 'value': '44444'}]}

In [26]:
data2 = [{'title': 'DEBT RECOVERY APPELLATE TRIBUNAL - ALLAHABAD', 'case_type': [{'name': 'REVIEW APPLICATION', 'value': '2'}, {'name': 'MISC APPLICATION', 'value': '3'}, {'name': 'REGULAR APPEAL', 'value': '4'}, {'name': 'CHAMBER APPEAL', 'value': '12'}, {'name': 'MISC APPEAL', 'value': '13'}, {'name': 'IA', 'value': '14'}, {'name': 'TRANSFER PETITION', 'value': '15'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '101'}, 'type': 'drt', 'cli_slug': 'drt', 'unique_id': 'drat-allahabad'}, {'title': 'DEBT RECOVERY APPELLATE TRIBUNAL - CHENNAI', 'case_type': [{'name': 'REVIEW APPLICATION', 'value': '2'}, {'name': 'MISC APPLICATION', 'value': '3'}, {'name': 'REGULAR APPEAL', 'value': '4'}, {'name': 'CHAMBER APPEAL', 'value': '12'}, {'name': 'MISC APPEAL', 'value': '13'}, {'name': 'IA', 'value': '14'}, {'name': 'TRANSFER PETITION', 'value': '15'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '102'}, 'type': 'drt', 'cli_slug': 'drt', 'unique_id': 'drat-chennai'}, {'title': 'DEBT RECOVERY APPELLATE TRIBUNAL - DELHI', 'case_type': [{'name': 'REVIEW APPLICATION', 'value': '2'}, {'name': 'MISC APPLICATION', 'value': '3'}, {'name': 'REGULAR APPEAL', 'value': '4'}, {'name': 'CHAMBER APPEAL', 'value': '12'}, {'name': 'MISC APPEAL', 'value': '13'}, {'name': 'IA', 'value': '14'}, {'name': 'TRANSFER PETITION', 'value': '15'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '100'}, 'type': 'drt', 'cli_slug': 'drt', 'unique_id': 'drat-delhi'}, {'title': 'DEBT RECOVERY APPELLATE TRIBUNAL - KOLKATA', 'case_type': [{'name': 'REVIEW APPLICATION', 'value': '2'}, {'name': 'MISC APPLICATION', 'value': '3'}, {'name': 'REGULAR APPEAL', 'value': '4'}, {'name': 'CHAMBER APPEAL', 'value': '12'}, {'name': 'MISC APPEAL', 'value': '13'}, {'name': 'IA', 'value': '14'}, {'name': 'TRANSFER PETITION', 'value': '15'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '104'}, 'type': 'drt', 'cli_slug': 'drt', 'unique_id': 'drat-kolkata'}, {'title': 'DEBT RECOVERY APPELLATE TRIBUNAL - MUMBAI', 'case_type': [{'name': 'REVIEW APPLICATION', 'value': '2'}, {'name': 'MISC APPLICATION', 'value': '3'}, {'name': 'REGULAR APPEAL', 'value': '4'}, {'name': 'CHAMBER APPEAL', 'value': '12'}, {'name': 'MISC APPEAL', 'value': '13'}, {'name': 'IA', 'value': '14'}, {'name': 'TRANSFER PETITION', 'value': '15'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '103'}, 'type': 'drt', 'cli_slug': 'drt', 'unique_id': 'drat-mumbai'}, {'title': 'DEBTS RECOVERY TRIBUNAL AHMEDABAD(DRT 1)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'CA', 'value': '14'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '9'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-ahmedabad-1'}, {'title': 'DEBTS RECOVERY TRIBUNAL AHMEDABAD(DRT 2)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'CA', 'value': '14'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '10'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-ahmedabad-2'}, {'title': 'DEBTS RECOVERY TRIBUNAL ALLAHABAD', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'CA', 'value': '14'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '11'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-allahabad'}, {'title': 'DEBTS RECOVERY TRIBUNAL AURANGABAD', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'CA', 'value': '14'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '12'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-aurangabad'}, {'title': 'DEBTS RECOVERY TRIBUNAL BANGALORE (DRT 1)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Original Application', 'value': '14'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '13'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-bangalore-1'}, {'title': 'DEBTS RECOVERY TRIBUNAL BANGALORE (DRT 2)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '39'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-bangalore-2'}, {'title': 'DEBTS RECOVERY TRIBUNAL CHANDIGARH (DRT 1)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'CA', 'value': '14'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Execution', 'value': '16'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '14'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-chandigarh-1'}, {'title': 'DEBTS RECOVERY TRIBUNAL CHANDIGARH (DRT 2)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'CA', 'value': '14'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '15'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-chandigarh-2'}, {'title': 'DEBTS RECOVERY TRIBUNAL CHANDIGARH (DRT 3)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'CA', 'value': '14'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '40'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-chandigarh-3'}, {'title': 'DEBTS RECOVERY TRIBUNAL CHENNAI (DRT 1)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '16'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-chennai-1'}, {'title': 'DEBTS RECOVERY TRIBUNAL CHENNAI (DRT 2)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '17'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-chennai-2'}, {'title': 'DEBTS RECOVERY TRIBUNAL CHENNAI (DRT 3)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '18'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-chennai-3'}, {'title': 'DEBTS RECOVERY TRIBUNAL COIMBATORE', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '19'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-coimbatore'}, {'title': 'DEBTS RECOVERY TRIBUNAL CUTTACK', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '20'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-cuttack'}, {'title': 'DEBTS RECOVERY TRIBUNAL DEHRADUN', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Execution', 'value': '13'}, {'name': 'CA', 'value': '14'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '41'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-dehradun'}, {'title': 'DEBTS RECOVERY TRIBUNAL DELHI(DRT 1)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Chamber Appeal', 'value': '14'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Execution', 'value': '16'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '1'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-delhi-1'}, {'title': 'DEBTS RECOVERY TRIBUNAL DELHI(DRT 2)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Execution', 'value': '13'}, {'name': 'Chamber Appeal', 'value': '14'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Exceution', 'value': '16'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}, {'name': 'Transfer Recovery Certificate', 'value': '24'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '2'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-delhi-2'}, {'title': 'DEBTS RECOVERY TRIBUNAL DELHI(DRT 3)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Chamber Appeal', 'value': '14'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Review Application', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '3'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-delhi-3'}, {'title': 'DEBTS RECOVERY TRIBUNAL ERNAKULAM(DRT 1)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '21'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-ernakulam-1'}, {'title': 'DEBTS RECOVERY TRIBUNAL ERNAKULAM(DRT 2)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '42'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-ernakulam-2'}, {'title': 'DEBTS RECOVERY TRIBUNAL GUWAHATI', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '22'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-guwahati'}, {'title': 'DEBTS RECOVERY TRIBUNAL HYDERABAD(DRT 1)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '23'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-hyderabad-1'}, {'title': 'DEBTS RECOVERY TRIBUNAL HYDERABAD(DRT 2)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '43'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-hyderabad-2'}, {'title': 'DEBTS RECOVERY TRIBUNAL JABALPUR', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Execution', 'value': '13'}, {'name': 'Chamber Appeal', 'value': '14'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'TAEX', 'value': '16'}, {'name': 'TEX', 'value': '17'}, {'name': 'OAEX', 'value': '18'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '24'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-jabalpur'}, {'title': 'DEBTS RECOVERY TRIBUNAL JAIPUR', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '25'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-jaipur'}, {'title': 'DEBTS RECOVERY TRIBUNAL KOLKATA(DRT 1)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '26'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-kolkata-1'}, {'title': 'DEBTS RECOVERY TRIBUNAL KOLKATA(DRT 2)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '27'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-kolkata-2'}, {'title': 'DEBTS RECOVERY TRIBUNAL KOLKATA(DRT 3)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '28'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-kolkata-3'}, {'title': 'DEBTS RECOVERY TRIBUNAL LUCKNOW', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Chamber Appeal', 'value': '14'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '29'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-lucknow'}, {'title': 'DEBTS RECOVERY TRIBUNAL MADURAI', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '30'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-madurai'}, {'title': 'DEBTS RECOVERY TRIBUNAL MUMBAI (DRT 1)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'CA', 'value': '14'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '31'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-mumbai-1'}, {'title': 'DEBTS RECOVERY TRIBUNAL MUMBAI (DRT 2)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'CA', 'value': '14'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '32'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-mumbai-2'}, {'title': 'DEBTS RECOVERY TRIBUNAL MUMBAI (DRT 3)', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'CA', 'value': '14'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '33'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-mumbai-3'}, {'title': 'DEBTS RECOVERY TRIBUNAL NAGPUR', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'CA', 'value': '14'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '34'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-nagpur'}, {'title': 'DEBTS RECOVERY TRIBUNAL PATNA', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'CA', 'value': '14'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '35'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-patna'}, {'title': 'DEBTS RECOVERY TRIBUNAL PUNE', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '36'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-pune'}, {'title': 'DEBTS RECOVERY TRIBUNAL RANCHI', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'CA', 'value': '14'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '37'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-ranchi'}, {'title': 'DEBTS RECOVERY TRIBUNAL SILIGURI', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Execution', 'value': '13'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '44'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-siliguri'}, {'title': 'DEBTS RECOVERY TRIBUNAL VISHAKHAPATNAM', 'case_type': [{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber appeal', 'value': '23'}], 'enabled': True, 'metadata': {'url': 'https://drt.etribunals.gov.in/front/drt_case_status.php', 'value': '38'}, 'type': 'drat', 'cli_slug': 'drat', 'unique_id': 'drt-vishakhapatnam'}]


In [21]:

for d in data2:
    if data1.get(d['unique_id']):
        print(d['case_type'])
        f.write("available for court"+d['unique_id']+"\n")
    else:
        f.write("not available for court"+d['unique_id']+"\n")
        print("not available for court",d['unique_id'],"\n")    


ValueError: I/O operation on closed file.

In [28]:
with open("rcRtc.txt", "w+") as f:
    for d in data2:
            if data1.get(d['unique_id']):
                print(d['case_type'])
                d['case_type'] = d['case_type']+ data1.get(d['unique_id'])
            else:
                print("not available for court",d['unique_id'],"\n")   
    f.write(str(data2))

not available for court drat-allahabad 

not available for court drat-chennai 

not available for court drat-delhi 

not available for court drat-kolkata 

not available for court drat-mumbai 

[{'name': 'Original Application', 'value': '1'}, {'name': 'Review Application', 'value': '2'}, {'name': 'Misc Application', 'value': '3'}, {'name': 'Appeal', 'value': '4'}, {'name': 'URA', 'value': '5'}, {'name': 'Transfer Application', 'value': '6'}, {'name': 'Securitization Application', 'value': '7'}, {'name': 'AIR', 'value': '8'}, {'name': 'COUNTER CLAIM', 'value': '10'}, {'name': 'Transfer Securitization Application', 'value': '11'}, {'name': 'Transfer Recovery Certificate', 'value': '12'}, {'name': 'Execution', 'value': '13'}, {'name': 'CA', 'value': '14'}, {'name': 'Transfer Miscellaneous Application', 'value': '15'}, {'name': 'Transfer Regular Appeal', 'value': '20'}, {'name': 'Transfer Appeal', 'value': '21'}, {'name': 'Transfer counter claim', 'value': '22'}, {'name': 'Transfer chamber